In [ ]:
import pandas as pd
import altair as alt

In [ ]:
alt.data_transformers.disable_max_rows()
url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(url)

In [ ]:
df.head(5)

In [ ]:
column_names = [
    "date_time",
    "city",
    "state",
    "country",
    "shape",
    "duration_seconds",
    "duration_text",
    "comments",
    "posted_date",
    "latitude",
    "longitude"
]
df = pd.read_csv(url, header=None, names=column_names)
df["date_time"] = pd.to_datetime(df["date_time"], errors="coerce")
df["posted_date"] = pd.to_datetime(df["posted_date"], errors="coerce")
df.head()

In [ ]:
df["year"] = df["date_time"].dt.year
df = df.dropna(subset=["latitude", "longitude"])
df = df[(df["latitude"] != 0) & (df["longitude"] != 0)]

In [ ]:
year_counts = df.groupby("year").size().reset_index(name="count")

plot1 = (
    alt.Chart(year_counts)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("count:Q", title="Number of UFO Reports"),
        tooltip=["year", "count"]
    )
    .properties(
        title="UFO Sightings Per Year",
        width=600,
        height=300
    )
)

plot1.save("plot1.json")
plot1

In [ ]:
state_dropdown = alt.binding_select(options=sorted(df["state"].dropna().unique()))
state_selection = alt.selection_point(fields=["state"], bind=state_dropdown, name="Select State")

plot2 = (
    alt.Chart(df)
    .mark_circle(size=60)
    .encode(
        x=alt.X("longitude:Q", title="Longitude"),
        y=alt.Y("latitude:Q", title="Latitude"),
        color=alt.Color("shape:N", title="UFO Shape"),
        tooltip=["city", "state", "date_time", "shape"]
    )
    .add_params(state_selection)
    .transform_filter(state_selection)
    .properties(
        title="UFO Reports by Location",
        width=600,
        height=400
    ).interactive()
)
plot2.save("plot2.json")
plot2